In [ ]:
import pandas as pd
import numpy as np
from equity_plot import equity_plot
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from datetime import datetime
from itertools import compress
from scipy import stats
# import qgrid
import yfinance as yf
import datetime as dt
import time
from progressbar import *
from ipywidgets import interact
import random
import pandas_market_calendars as mcal
import bisect 
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] 
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('ggplot')
# plt.rcParams['font.sans-serif'] = ['SimHei']
pd.set_option('display.max_columns', None)
pd.options.display.max_rows = 30

Data

In [ ]:
stock_price = pd.read_csv(r"C:\Users\qwtl0\Python\TMBA_data\TEJ_調整後股價資料\收盤價(元).csv")
stock_lend = pd.read_csv(r"C:\Users\qwtl0\Python\TMBA_data\TEJ_融資融券\融券(買+賣)除以成交量 %.csv")
stock_market = pd.read_csv(r"C:\Users\qwtl0\Python\TMBA_data\TEJ_借券餘額\市場別.csv")
stock_volume = pd.read_csv(r"C:\Users\qwtl0\Python\TMBA_data\TEJ_調整後股價資料\成交值(千元).csv")
industry = pd.read_csv(r"C:\Users\qwtl0\Python\TMBA_data\industry.csv")

Time

In [ ]:
bt_start = "2012-01-01"
bt_end = "2021-10-01"

Data Preprocessed

In [ ]:
lend_df = stock_lend.copy()
lend_df['年月日'] = pd.to_datetime(lend_df['年月日'].astype('str'))
lend_df = lend_df.set_index('年月日').dropna(axis = 1)
lend_df = lend_df.loc[bt_start:bt_end].replace(['           -','      -','     -','              -','        -'],np.nan).dropna(axis = 1).astype('float')

stock_df = stock_price.copy()
stock_df['年月日'] = pd.to_datetime(stock_df['年月日'].astype('str'))
stock_df = stock_df.set_index('年月日').dropna(axis = 1)
stock_df = stock_df.loc[bt_start:bt_end]

market_df = stock_market.copy()
market_df['年月日'] = pd.to_datetime(market_df['年月日'].astype('str'))
market_df = market_df.set_index('年月日')
market_df = market_df.loc[bt_start:bt_end].dropna(axis = 1)

volume_df = stock_volume.copy()
volume_df['年月日'] = pd.to_datetime(volume_df['年月日'].astype('str'))
volume_df = volume_df.set_index('年月日').dropna(axis = 1)
volume_df = volume_df.loc[bt_start:bt_end]

industry['公司簡稱'] = industry['公司簡稱'].astype(str)
industry_df = industry.copy()

stock_set = set(stock_df.columns) & set(lend_df.columns)&set(market_df.tail(1)[market_df.tail(1) == 'TSE  '].dropna(axis = 1).columns)
stock_df = stock_df[list(stock_set)]
lend_df = lend_df[list(stock_set)]
stock_close = stock_df.copy()

Signal

In [ ]:
signal_df = lend_df

Condition

In [ ]:
def stock_CTA_signal(rebalance_day,data,condition1,condition2) :
    df = data.loc[rebalance_day][condition1 & condition2]
    signal = df.keys()
    return signal

Rebalance Day

In [ ]:
start_list = []
rebalance_day = 1
for x in range(len(stock_df.index)) :
    if x%rebalance_day == 0:
        start_list.append(stock_df.index[x])
        
end_list = []
for x in range(len(stock_df.index)-(rebalance_day-1)) :
    if x%rebalance_day == 0:
        end_list.append(stock_df.index[x+(rebalance_day-1)])


In [ ]:
rebalance_list = {'start' : start_list[:len(end_list)],'end' : end_list}
pd.DataFrame(rebalance_list)

CTA Backtest

In [ ]:
ret_long_list = []
tic_long_list = []

cost = -0.001425

data1 = signal_df
# data1 = lend_df
# data2 = stock_df
# data3 = stock_21day_high

start_time = time.time()
for i in range(1,len(rebalance_list['start'])) :

    #     訊號
    long_signal = stock_CTA_signal(rebalance_list['end'][i-1],data = data1, 
                                   condition1 = signal_df.loc[rebalance_list['end'][i-1]] > 0.06 , 
                                   condition2 = (lend_df.loc[rebalance_list['end'][i-1]] < 0.03) & (lend_df.loc[rebalance_list['end'][i-1]] > 0) )
    if long_signal.values.squeeze() != [] : 
        print(rebalance_list['end'][i-1]," 進場 ",long_signal.values.squeeze())
    #     回測   
        ret_long = stock_close.loc[rebalance_list['start'][i]:rebalance_list['end'][i],long_signal].pct_change().fillna(0).T.mean()
    
        tic_long = stock_close.loc[rebalance_list['start'][i]:rebalance_list['end'][i],long_signal].columns   
        
        ret_long[0] = cost
    
    else :
        ret_long = stock_close.loc[rebalance_list['start'][i]:rebalance_list['end'][i],long_signal].pct_change().fillna(0).T.mean()
    
        tic_long = stock_close.loc[rebalance_list['start'][i]:rebalance_list['end'][i],long_signal].columns 
        
        

    
    ret_long_list.append(ret_long)
    tic_long_list.append(tic_long)

ret_long_df = pd.DataFrame(pd.DataFrame(ret_long_list).fillna(0).sum())
tic_long_df = pd.DataFrame(tic_long_list)

end_time = time.time()
print('此次回測總共耗時 %f 秒' %(round(end_time-start_time,2)))